<a href="https://colab.research.google.com/github/Smarth-Bansal/Formula-generator/blob/main/Formula_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Define the template for the HTML file
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Solution Of Triangles</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #f5f5f5;
            margin: 0;
            padding: 0;
            overflow: hidden; /* Prevent scrolling while overlay is visible */
        }}

        .container {{
            max-width: 600px;
            margin: 20px auto;
            padding: 20px;
            background-color: #fff;
            border-radius: 8px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }}

        h1 {{
            text-align: center;
            color: #333;
        }}

        table {{
            width: 100%;
            border-collapse: collapse;
        }}

        th, td {{
            padding: 10px;
            text-align: left;
        }}

        th {{
            background-color: #4CAF50;
            color: white;
        }}

        td {{
            border-bottom: 1px solid #ddd;
        }}

        .toggle-btn {{
            background-color: #4CAF50;
            color: white;
            border: none;
            padding: 5px 10px;
            cursor: pointer;
            border-radius: 4px;
        }}

        .toggle-btn:hover {{
            background-color: #45a049;
        }}

        @media (max-width: 600px) {{
            table {{
                border: 0;
            }}

            thead {{
                display: none;
            }}

            tr {{
                display: block;
                margin-bottom: 10px;
            }}

            td {{
                display: block;
                text-align: right;
                font-size: 14px;
                position: relative;
                padding-left: 50%;
            }}

            td::before {{
                content: attr(data-label);
                position: absolute;
                left: 0;
                width: 50%;
                padding-left: 10px;
                font-weight: bold;
                text-align: left;
            }}
        }}

        /* Full-screen overlay styles */
        .overlay {{
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background: rgba(0, 0, 0, 0.8);
            display: flex;
            align-items: center;
            justify-content: center;
            z-index: 2000;
            transition: opacity 0.5s ease;
            opacity: 1;
            visibility: visible;
            color: white;
            text-align: center;
            font-size: 24px;
            font-weight: bold;
            flex-direction: column;
        }}

        .overlay.hidden {{
            opacity: 0;
            visibility: hidden;
        }}

        .spinner {{
            border: 16px solid #f3f3f3;
            border-top: 16px solid #3498db;
            border-radius: 50%;
            width: 120px;
            height: 120px;
            animation: spin 1s linear infinite;
        }}

        @keyframes spin {{
            0% {{ transform: rotate(0deg); }}
            100% {{ transform: rotate(360deg); }}
        }}

        .message {{
            margin-bottom: 20px;
        }}
    </style>
    <script id="MathJax-script" async src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js"></script>
    <script>
        function checkOrientation() {{
            const overlay = document.querySelector('.overlay');
            if (window.innerWidth <= window.innerHeight) {{
                overlay.classList.remove('hidden');
                document.body.style.overflow = 'hidden'; // Prevent scrolling while overlay is visible
            }} else {{
                overlay.classList.add('hidden');
                document.body.style.overflow = 'auto'; // Re-enable scrolling
            }}
        }}

        window.addEventListener('resize', checkOrientation);
        window.addEventListener('orientationchange', checkOrientation);

        // Initial check
        document.addEventListener('DOMContentLoaded', checkOrientation);
    </script>
</head>
<body>
    <div class="overlay">
        <div class="message">Please turn your screen to landscape mode for the best experience.</div>
        <div class="spinner"></div>
    </div>
    <div class="container">
        <h1>Solution Of Triangles</h1>
        <table id="formulaTable">
            <thead>
                <tr>
                    <th>Formula Name</th>
                    <th>Show/Hide</th>
                    <th>Formula</th>
                </tr>
            </thead>
            <tbody>
                {rows}
            </tbody>
        </table>
    </div>

    <script>
        function toggleFormula(button) {{
            const formulaCell = button.parentElement.nextElementSibling;
            if (formulaCell.style.display === 'none') {{
                formulaCell.style.display = 'table-cell';
                button.textContent = 'Hide';
                MathJax.typeset();
            }} else {{
                formulaCell.style.display = 'none';
                button.textContent = 'Show';
            }}
        }}
    </script>
</body>
</html>
'''
# Define the formula data
formulas = [
    {"name": "Law Of Sines", "formula": "\\( \\frac{a}{\\sin A} = \\frac{b}{\\sin B} = \\frac{c}{\\sin C} = 2R \\)"},
    {"name": "Side-Angle Relations", "formula": "\\( a = 2R \\sin A \\)"},
    {"name": "Law of Cosines", "formula": "\\( a^2 = b^2 + c^2 - 2bc \\cos A \\)"},
    {"name": "Napier's Analogy (Tangent Rule)", "formula": "\\( \\tan \\frac{B - C}{2} = \\frac{b - c}{b + c} \\cot \\frac{A}{2} \\)"},
    {"name": "Half Angle Formula (Sine)", "formula": "\\( \\sin \\frac{A}{2} = \\sqrt{\\frac{(s - b)(s - c)}{bc}} \\)"},
    {"name": "Half Angle Formula (Cosine)", "formula": "\\( \\cos \\frac{A}{2} = \\sqrt{\\frac{(s(s - a))}{bc}} \\)"},
    {"name": "Half Angle Formula (Tangent)", "formula": "\\( \\tan \\frac{A}{2} = \\sqrt{\\frac{(s - b)(s - c)}{s(s - a)}} = \\frac{(s - b)(s - c)}{\\Delta} \\)"},
    {"name": "Semi-Perimeter", "formula": "\\( s = 4R \\cos \\frac{A}{2} \\cos \\frac{B}{2} \\cos \\frac{C}{2} \\)"},
    {"name": "Area (Heron's Formula)", "formula": "\\( \\Delta = \\sqrt{s(s - a)(s - b)(s - c)} \\)"},
    {"name": "Area (Side-Angle Formula)", "formula": "\\( \\Delta = \\frac{1}{2} a b \\sin C \\)"},
    {"name": "Area (Side-Circumcenter Formula)", "formula": "\\( \\Delta = \\frac{a b c}{4 R} \\)"},
    {"name": "Area (Angle-Circumcenter Formula)", "formula": "\\( \\Delta = 2 R^2 \\sin A \\sin B \\sin C \\)"},
    {"name": "Area (Incenter-Semi-perimeter Formula)", "formula": "\\( \\Delta = r s \\)"},
    {"name": "Side of a Circumscribed n-sided Polygon", "formula": "\\( \\text{Side} = 2r \\tan \\frac{\\pi}{n} \\)"},
    {"name": "Side of an Inscribed n-sided Polygon", "formula": "\\( \\text{Side} = 2r \\sin \\frac{\\pi}{n} \\)"},
    {"name": "Area of an Inscribed n-sided Polygon", "formula": "\\( \\Delta = \\frac{1}{2} nr^2 \\sin \\frac{2\\pi}{n} \\)"},
    {"name": "Area of a Circumscribed n-sided Polygon", "formula": "\\( \\Delta = nr^2 \\tan \\frac{\\pi}{n} \\)"},
    {"name": "Incenter (Angle-Circumcenter Formula)", "formula": "\\( r = 4R \\sin \\frac{A}{2} \\sin \\frac{B}{2} \\sin \\frac{C}{2} \\)"},
    {"name": "Incenter (Angle-Side Formula)", "formula": "\\( r = (s - a) \\tan \\frac{A}{2} \\)"},
]

# Generate table rows
rows = ""
for formula in formulas:
    if formula["name"] and formula["formula"]:  # Only include non-empty rows
        rows += f'''
        <tr>
            <td>{formula["name"]}</td>
            <td><button class="toggle-btn" onclick="toggleFormula(this)">Show</button></td>
            <td class="formula-cell" style="display: none;">{formula["formula"]}</td>
        </tr>
        '''

# Write the HTML file
html_content = HTML_TEMPLATE.format(rows=rows)

# Save to an HTML file
output_file = "SOT.html"
with open(output_file, "w") as file:
    file.write(html_content)